In [53]:
install.packages("crate")

Installing package into 'C:/Users/Admin/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)

Warning message:
"package 'crate' is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages"


In [33]:
install.packages("xgboost")

Installing package into 'C:/Users/Admin/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)

also installing the dependency 'data.table'




package 'data.table' successfully unpacked and MD5 sums checked
package 'xgboost' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Admin\AppData\Local\Temp\RtmpMPGTdl\downloaded_packages


In [34]:
library(mlflow)
library(MASS)
library(rpart)
library(randomForest)
library(xgboost)

In [32]:
data("Boston")
set.seed(123)
train_idx <- sample(1:nrow(Boston), 0.8 * nrow(Boston))
train <- Boston[train_idx, ]
test <- Boston[-train_idx, ]

In [15]:
mlflow_set_tracking_uri("http://127.0.0.1:5000")
mlflow_set_experiment("MLflow_Quickstart_R")

Experiment `MLflow_Quickstart_R` does not exist. Creating a new experiment.



[1] "335984739251647557"

In [10]:
rmse <- function(actual, predicted) sqrt(mean((actual - predicted)^2))

In [19]:
# -----------------------------
# Model 1: Linear Regression
# -----------------------------
model_lm <- lm(medv ~ ., data = train)
pred_lm <- predict(model_lm, newdata = test)
mse_lm <- mean((pred_lm - test$medv)^2)
r_squared_lm <- 1 - sum((pred_lm - test$medv)^2) / sum((test$medv - mean(test$medv))^2)
# mlflow_save_model
mlflow_start_run()
mlflow_log_param("model_type", "linear_regression")
mlflow_log_metric("mse", mse_lm)
mlflow_log_metric("r_squared", r_squared_lm)
mlflow_log_model(model_lm, "model")  # logs R model
mlflow_end_run()

ERROR: Error: Run with UUID 56faffcbe7be47e88b338862723ae722 is already active. To start a nested run, Call `mlflow_start_run()` with `nested = TRUE`.


In [26]:
model_tree <- rpart(medv ~ ., data = train)
pred_tree <- predict(model_tree, newdata = test)
mse_tree <- mean((pred_tree - test$medv)^2)
r_squared_tree <- 1 - sum((pred_tree - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

mlflow_start_run()
mlflow_log_param("model_type", "decision_tree")
mlflow_log_metric("mse", mse_tree)
mlflow_log_metric("r_squared", r_squared_tree)
mlflow_log_model(model_tree, "model")
mlflow_end_run()

run_uuid                         experiment_id      run_name       user_id
1 d96e67d8692a474b815d9260dfb5761f 335984739251647557 gentle-doe-766 Admin  
2 d96e67d8692a474b815d9260dfb5761f 335984739251647557 gentle-doe-766 Admin  
3 d96e67d8692a474b815d9260dfb5761f 335984739251647557 gentle-doe-766 Admin  
4 d96e67d8692a474b815d9260dfb5761f 335984739251647557 gentle-doe-766 Admin  
  status  start_time         
1 RUNNING 2025-09-23 16:09:38
2 RUNNING 2025-09-23 16:09:38
3 RUNNING 2025-09-23 16:09:38
4 RUNNING 2025-09-23 16:09:38
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/d96e67d8692a474b815d9260dfb5761f/artifacts
2 mlflow-artifacts:/335984739251647557/d96e67d8692a474b815d9260dfb5761f/artifacts
3 mlflow-artifacts:/335984739251647557/d96e67d8692a474b815d9260dfb5761f/artifacts
4 mlflow-artifacts:/335984739251647557/d96e67d8692a474b815d9260dfb5761f/artifacts
  lifecycle_stage run_id                           end_time metrics params
1 active          d96e67d8692a474b815d9260dfb5761f NA       NA      NA    
2 active          d96e67d8692a474b815d9260dfb5761f NA       NA      NA    
3 active          d96e67d8692a474b815d9260dfb5761f NA       NA      NA    
4 active          d96e67d8692a474b815d9260dfb5761f NA       NA      NA    
  tags               <NA>          
1 mlflow.user        Admin         
2 mlflow.source.name <console>     
3 mlflow.runName     gentle-doe-766
4 mlflow.source.type LOCAL

ERROR: Error in UseMethod("mlflow_save_model"): no applicable method for 'mlflow_save_model' applied to an object of class "rpart"


In [50]:
model_rf <- randomForest(medv ~ ., data = train)
pred_rf <- predict(model_rf, newdata = test)
mse_rf <- mean((pred_rf - test$medv)^2)
r_squared_rf <- 1 - sum((pred_rf - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

mlflow_start_run()
mlflow_log_param("model_type", "random_forest")
mlflow_log_metric("mse", mse_rf)
mlflow_log_metric("r_squared", r_squared_rf)
rf_predict_wrapper <- function(df) {
  predict(model_rf, df)
}

custom_rf <- mlflow_rfunc(
  func = rf_predict_wrapper,
  conda_env = mlflow_default_conda_env()
)

mlflow_save_model(custom_rf, path = "rf_custom_model")
mlflow_log_model("rf_custom_model", "model")
mlflow_end_run()

run_uuid                         experiment_id      run_name         user_id
1 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
2 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
3 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
4 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
  status  start_time         
1 RUNNING 2025-09-23 17:27:18
2 RUNNING 2025-09-23 17:27:18
3 RUNNING 2025-09-23 17:27:18
4 RUNNING 2025-09-23 17:27:18
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
2 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
3 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
4 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
  lifecycle_stage run_id                           end_time metrics params
1 active          75e58d07d66e422b9a5c568f68d27c99 NA       NA      NA    
2 active          75e58d07d66e422b9a5c568f68d27c99 NA       NA      NA    
3 active          75e58d07d66e422b9a5c568f68d27c99 NA       NA      NA    
4 active          75e58d07d66e422b9a5c568f68d27c99 NA       NA      NA    
  tags               <NA>            
1 mlflow.user        Admin           
2 mlflow.source.name <console>       
3 mlflow.runName     likeable-cod-568
4 mlflow.source.type LOCAL

ERROR: Error in mlflow_rfunc(func = rf_predict_wrapper, conda_env = mlflow_default_conda_env()): could not find function "mlflow_rfunc"


In [51]:
mlflow_end_run()

run_uuid                         experiment_id      run_name         user_id
1 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
2 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
3 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
4 75e58d07d66e422b9a5c568f68d27c99 335984739251647557 likeable-cod-568 Admin  
  status   start_time          end_time           
1 FINISHED 2025-09-23 17:27:18 2025-09-23 17:28:33
2 FINISHED 2025-09-23 17:27:18 2025-09-23 17:28:33
3 FINISHED 2025-09-23 17:27:18 2025-09-23 17:28:33
4 FINISHED 2025-09-23 17:27:18 2025-09-23 17:28:33
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
2 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
3 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
4 mlflow-artifacts:/335984739251647557/75e58d07d66e422b9a5c568f68d27c99/artifacts
  lifecycle_stage run_id                           metrics   <NA>      
1 active          75e58d07d66e422b9a5c568f68d27c99 mse       11.8620566
2 active          75e58d07d66e422b9a5c568f68d27c99 r_squared  0.8594259
3 active          75e58d07d66e422b9a5c568f68d27c99 mse       11.8620566
4 active          75e58d07d66e422b9a5c568f68d27c99 r_squared  0.8594259
  <NA>                <NA> <NA>                             params    
1 2025-09-23 17:27:18 0    75e58d07d66e422b9a5c568f68d27c99 model_type
2 2025-09-23 17:27:18 0    75e58d07d66e422b9a5c568f68d27c99 model_type
3 2025-09-23 17:27:18 0    75e58d07d66e422b9a5c568f68d27c99 model_type
4 2025-09-23 17:27:18 0    75e58d07d66e422b9a5c568f68d27c99 model_type
  <NA>          tags               <NA>            
1 random_forest mlflow.user        Admin           
2 random_forest mlflow.source.name <console>       
3 random_forest mlflow.runName     likeable-cod-568
4 random_forest mlflow.source.type LOCAL

In [35]:
###  XGBoost Data Prepartion 
train_matrix <- as.matrix(train[,-which(names(train) == "medv")])
train_label <- train$medv
test_matrix <- as.matrix(test[,-which(names(test) == "medv")])

In [42]:
# Train XGBoost model v1
xgb_model_v1 <- xgboost(
  data = train_matrix,
  label = train_label,
  nrounds = 100,
  max_depth = 4,
  eta = 0.1,
  subsample = 0.8,
  colsample_bytree = 0.8,
  objective = "reg:squarederror",
  verbose = 0
)

### 
mlflow_start_run()
mlflow_log_param("model_type", "xgboost_v1")

# Log hyperparameters
mlflow_log_param("nrounds", 100)
mlflow_log_param("max_depth", 4)
mlflow_log_param("eta", 0.1)
mlflow_log_param("subsample", 0.8)
mlflow_log_param("colsample_bytree", 0.8)
mlflow_log_param("objective", "reg:squarederror")



ERROR: Error: Run with UUID d1f5f91ee91343638bb618a96c8949c8 is already active. To start a nested run, Call `mlflow_start_run()` with `nested = TRUE`.


In [43]:
pred_xgb_v1 <- predict(xgb_model_v1, test_matrix)
mse_xgb_v1 <- mean((pred_xgb_v1 - test$medv)^2)
r_squared_xgb_v1 <- 1 - sum((pred_xgb_v1 - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

mlflow_log_metric("mse", mse_xgb_v1)
mlflow_log_metric("r_squared", r_squared_xgb_v1)

# Log the model artifact
mlflow_log_model(xgb_model_v1, "model")



Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/23 16:32:56 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpMPGTdl/model to artifact_path=model


Root URI: mlflow-artifacts:/335984739251647557/d1f5f91ee91343638bb618a96c8949c8/artifacts

Warning message in value[[3L]](cond):
"Logging model metadata to the tracking server has failed, possibly due to older server version. The model artifacts have been logged successfully. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above."


path,is_dir,file_size
<chr>,<lgl>,<int>
model/MLmodel,FALSE,327
model/conda_env.yaml,FALSE,93
model/model.xgb,FALSE,101775
model/python_env.yaml,FALSE,92


In [44]:
mlflow_end_run()

run_uuid                         experiment_id      run_name           
1 d1f5f91ee91343638bb618a96c8949c8 335984739251647557 thundering-crow-821
2 d1f5f91ee91343638bb618a96c8949c8 335984739251647557 thundering-crow-821
3 d1f5f91ee91343638bb618a96c8949c8 335984739251647557 thundering-crow-821
4 d1f5f91ee91343638bb618a96c8949c8 335984739251647557 thundering-crow-821
  user_id status   start_time          end_time           
1 Admin   FINISHED 2025-09-23 16:31:56 2025-09-23 16:32:56
2 Admin   FINISHED 2025-09-23 16:31:56 2025-09-23 16:32:56
3 Admin   FINISHED 2025-09-23 16:31:56 2025-09-23 16:32:56
4 Admin   FINISHED 2025-09-23 16:31:56 2025-09-23 16:32:56
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/d1f5f91ee91343638bb618a96c8949c8/artifacts
2 mlflow-artifacts:/335984739251647557/d1f5f91ee91343638bb618a96c8949c8/artifacts
3 mlflow-artifacts:/335984739251647557/d1f5f91ee91343638bb618a96c8949c8/artifacts
4 mlflow-artifacts:/335984739251647557/d1f5f91ee91343638bb618a96c8949c8/artifacts
  lifecycle_stage run_id                           metrics   <NA>      
1 active          d1f5f91ee91343638bb618a96c8949c8 mse       13.7718782
2 active          d1f5f91ee91343638bb618a96c8949c8 r_squared  0.8367931
3 active          d1f5f91ee91343638bb618a96c8949c8 mse       13.7718782
4 active          d1f5f91ee91343638bb618a96c8949c8 r_squared  0.8367931
  <NA>                <NA> <NA>                             params
1 2025-09-23 16:32:45 0    d1f5f91ee91343638bb618a96c8949c8 NA    
2 2025-09-23 16:32:45 0    d1f5f91ee91343638bb618a96c8949c8 NA    
3 2025-09-23 16:32:45 0    d1f5f91ee91343638bb618a96c8949c8 NA    
4 2025-09-23 16:32:45 0    d1f5f91ee91343638bb618a96c8949c8 NA    
  tags               <NA>               
1 mlflow.user        Admin              
2 mlflow.source.name <console>          
3 mlflow.runName     thundering-crow-821
4 mlflow.source.type LOCAL

In [47]:
xgb_model_v2 <- xgboost(
  data = train_matrix,
  label = train_label,
  nrounds = 200,
  max_depth = 6,
  eta = 0.05,               # smaller learning rate → slower, more precise
  subsample = 0.9,
  colsample_bytree = 0.9,
  min_child_weight = 3,
  gamma = 1,
  objective = "reg:squarederror",
  verbose = 0
)
mlflow_start_run()

# Log model type
mlflow_log_param("model_type", "xgboost_v2")
mlflow_log_param("nrounds", 100)
mlflow_log_param("max_depth", 4)
mlflow_log_param("eta", 0.1)
mlflow_log_param("subsample", 0.8)
mlflow_log_param("colsample_bytree", 0.8)
mlflow_log_param("objective", "reg:squarederror")
pred_xgb_v1 <- predict(xgb_model_v1, test_matrix)
mse_xgb_v1 <- mean((pred_xgb_v1 - test$medv)^2)
r_squared_xgb_v1 <- 1 - sum((pred_xgb_v1 - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

mlflow_log_metric("mse", mse_xgb_v1)
mlflow_log_metric("r_squared", r_squared_xgb_v1)

# Log the model artifact
mlflow_log_model(xgb_model_v1, "model")

mlflow_end_run()

run_uuid                         experiment_id      run_name          
1 16beaab6e8144a72a9335d1f8370aeb8 335984739251647557 honorable-bear-951
2 16beaab6e8144a72a9335d1f8370aeb8 335984739251647557 honorable-bear-951
3 16beaab6e8144a72a9335d1f8370aeb8 335984739251647557 honorable-bear-951
4 16beaab6e8144a72a9335d1f8370aeb8 335984739251647557 honorable-bear-951
  user_id status  start_time         
1 Admin   RUNNING 2025-09-23 16:40:25
2 Admin   RUNNING 2025-09-23 16:40:25
3 Admin   RUNNING 2025-09-23 16:40:25
4 Admin   RUNNING 2025-09-23 16:40:25
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/16beaab6e8144a72a9335d1f8370aeb8/artifacts
2 mlflow-artifacts:/335984739251647557/16beaab6e8144a72a9335d1f8370aeb8/artifacts
3 mlflow-artifacts:/335984739251647557/16beaab6e8144a72a9335d1f8370aeb8/artifacts
4 mlflow-artifacts:/335984739251647557/16beaab6e8144a72a9335d1f8370aeb8/artifacts
  lifecycle_stage run_id                           end_time metrics params
1 active          16beaab6e8144a72a9335d1f8370aeb8 NA       NA      NA    
2 active          16beaab6e8144a72a9335d1f8370aeb8 NA       NA      NA    
3 active          16beaab6e8144a72a9335d1f8370aeb8 NA       NA      NA    
4 active          16beaab6e8144a72a9335d1f8370aeb8 NA       NA      NA    
  tags               <NA>              
1 mlflow.user        Admin             
2 mlflow.source.name <console>         
3 mlflow.runName     honorable-bear-951
4 mlflow.source.type LOCAL

Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/23 16:40:37 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpMPGTdl/model to artifact_path=model


Root URI: mlflow-artifacts:/335984739251647557/16beaab6e8144a72a9335d1f8370aeb8/artifacts

Warning message in value[[3L]](cond):
"Logging model metadata to the tracking server has failed, possibly due to older server version. The model artifacts have been logged successfully. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above."


path,is_dir,file_size
<chr>,<lgl>,<int>
model/MLmodel,FALSE,327
model/conda_env.yaml,FALSE,93
model/model.xgb,FALSE,101775
model/python_env.yaml,FALSE,92


ERROR while rich displaying an object: Error in data.frame(..., check.names = FALSE): arguments imply differing number of rows: 1, 2, 7, 4

Traceback:
1. sapply(x, f, simplify = simplify)
2. lapply(X = X, FUN = FUN, ...)
3. FUN(X[[i]], ...)
4. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
5. tryCatchList(expr, classes, parentenv, handlers)
6. tryCatchOne(expr, names, parentenv, handlers[[1L]])
7. doTryCatch(return(expr), name, parentenv, handler)
8. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_conten

In [56]:

install.packages("carrier")


Installing package into 'C:/Users/Admin/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)

also installing the dependencies 'prettyunits', 'cpp11', 'lobstr'




package 'prettyunits' successfully unpacked and MD5 sums checked
package 'cpp11' successfully unpacked and MD5 sums checked
package 'lobstr' successfully unpacked and MD5 sums checked
package 'carrier' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Admin\AppData\Local\Temp\RtmpMPGTdl\downloaded_packages


In [58]:
install.packages("crate")

Installing package into 'C:/Users/Admin/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)

Warning message:
"package 'crate' is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages"


In [59]:
library(carrier)

In [63]:
model_rf <- randomForest(medv ~ ., data = train)
pred_rf <- predict(model_rf, newdata = test)
mse_rf <- mean((pred_rf - test$medv)^2)
r_squared_rf <- 1 - sum((pred_rf - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

# Start MLflow run
mlflow_start_run()
mlflow_log_param("model_type", "random_forest")
mlflow_log_metric("mse", mse_rf)
mlflow_log_metric("r_squared", r_squared_rf)

# Wrap prediction function + model inside crate
rf_predict_wrapper <- crate(~ predict(model, .x), model = model_rf)

# Log the crate as a model
mlflow_log_model(rf_predict_wrapper, "rf_custom_model")

mlflow_end_run()

run_uuid                         experiment_id      run_name           
1 300e7893a02842bda57328a344ecf2d6 335984739251647557 gregarious-mole-627
2 300e7893a02842bda57328a344ecf2d6 335984739251647557 gregarious-mole-627
3 300e7893a02842bda57328a344ecf2d6 335984739251647557 gregarious-mole-627
4 300e7893a02842bda57328a344ecf2d6 335984739251647557 gregarious-mole-627
  user_id status  start_time         
1 Admin   RUNNING 2025-09-23 17:41:14
2 Admin   RUNNING 2025-09-23 17:41:14
3 Admin   RUNNING 2025-09-23 17:41:14
4 Admin   RUNNING 2025-09-23 17:41:14
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/300e7893a02842bda57328a344ecf2d6/artifacts
2 mlflow-artifacts:/335984739251647557/300e7893a02842bda57328a344ecf2d6/artifacts
3 mlflow-artifacts:/335984739251647557/300e7893a02842bda57328a344ecf2d6/artifacts
4 mlflow-artifacts:/335984739251647557/300e7893a02842bda57328a344ecf2d6/artifacts
  lifecycle_stage run_id                           end_time metrics params
1 active          300e7893a02842bda57328a344ecf2d6 NA       NA      NA    
2 active          300e7893a02842bda57328a344ecf2d6 NA       NA      NA    
3 active          300e7893a02842bda57328a344ecf2d6 NA       NA      NA    
4 active          300e7893a02842bda57328a344ecf2d6 NA       NA      NA    
  tags               <NA>               
1 mlflow.user        Admin              
2 mlflow.source.name <console>          
3 mlflow.runName     gregarious-mole-627
4 mlflow.source.type LOCAL

Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/23 17:41:24 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpMPGTdl/rf_custom_model to artifact_path=rf_custom_model


Root URI: mlflow-artifacts:/335984739251647557/300e7893a02842bda57328a344ecf2d6/artifacts



path,is_dir,file_size
<chr>,<lgl>,<int>
rf_custom_model/MLmodel,FALSE,182
rf_custom_model/crate.bin,FALSE,1021374


ERROR while rich displaying an object: Error in data.frame(..., check.names = FALSE): arguments imply differing number of rows: 1, 2, 5

Traceback:
1. sapply(x, f, simplify = simplify)
2. lapply(X = X, FUN = FUN, ...)
3. FUN(X[[i]], ...)
4. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
5. tryCatchList(expr, classes, parentenv, handlers)
6. tryCatchOne(expr, names, parentenv, handlers[[1L]])
7. doTryCatch(return(expr), name, parentenv, handler)
8. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(i

In [64]:
mlflow_end_run()

ERROR: Error: There is no active run to end.


In [69]:

# Wrap your model in a crate
rf_predict_wrapper <- crate(~ predict(model_rf, .x), model = model_rf)

# Define manual input schema (column names + types must match your training data)
input_schema <- list(
  columns = list(
    list(name = "crim", type = "double"),
    list(name = "zn", type = "double"),
    list(name = "indus", type = "double"),
    list(name = "chas", type = "double"),
    list(name = "nox", type = "double"),
    list(name = "rm", type = "double"),
    list(name = "age", type = "double"),
    list(name = "dis", type = "double"),
    list(name = "rad", type = "double"),
    list(name = "tax", type = "double"),
    list(name = "ptratio", type = "double"),
    list(name = "black", type = "double"),
    list(name = "lstat", type = "double")
  )
)

# Define output schema
output_schema <- list(
  columns = list(
    list(name = "prediction", type = "double")
  )
)

# Start MLflow run
mlflow_start_run()

mlflow_log_param("model_type", "random_forest")

# Log the crate model with manual signature
mlflow_log_model(
  rf_predict_wrapper,
  artifact_path = "rf_custom_model",
  signature = list(inputs = input_schema, outputs = output_schema)
)

mlflow_end_run()


run_uuid                         experiment_id      run_name          
1 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
2 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
3 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
4 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
  user_id status  start_time         
1 Admin   RUNNING 2025-09-23 18:22:25
2 Admin   RUNNING 2025-09-23 18:22:25
3 Admin   RUNNING 2025-09-23 18:22:25
4 Admin   RUNNING 2025-09-23 18:22:25
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
2 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
3 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
4 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
  lifecycle_stage run_id                           end_time metrics params
1 active          c5ed16e8d0464810b84e679046358a78 NA       NA      NA    
2 active          c5ed16e8d0464810b84e679046358a78 NA       NA      NA    
3 active          c5ed16e8d0464810b84e679046358a78 NA       NA      NA    
4 active          c5ed16e8d0464810b84e679046358a78 NA       NA      NA    
  tags               <NA>              
1 mlflow.user        Admin             
2 mlflow.source.name <console>         
3 mlflow.runName     capricious-bug-290
4 mlflow.source.type LOCAL

Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/23 18:22:36 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpMPGTdl/rf_custom_model to artifact_path=rf_custom_model


Root URI: mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts



path,is_dir,file_size
<chr>,<lgl>,<int>
rf_custom_model/MLmodel,FALSE,182
rf_custom_model/crate.bin,FALSE,1021383


run_uuid                         experiment_id      run_name          
1 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
2 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
3 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
4 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
5 c5ed16e8d0464810b84e679046358a78 335984739251647557 capricious-bug-290
  user_id status   start_time          end_time           
1 Admin   FINISHED 2025-09-23 18:22:25 2025-09-23 18:22:37
2 Admin   FINISHED 2025-09-23 18:22:25 2025-09-23 18:22:37
3 Admin   FINISHED 2025-09-23 18:22:25 2025-09-23 18:22:37
4 Admin   FINISHED 2025-09-23 18:22:25 2025-09-23 18:22:37
5 Admin   FINISHED 2025-09-23 18:22:25 2025-09-23 18:22:37
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
2 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
3 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
4 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
5 mlflow-artifacts:/335984739251647557/c5ed16e8d0464810b84e679046358a78/artifacts
  lifecycle_stage run_id                           metrics params    
1 active          c5ed16e8d0464810b84e679046358a78 NA      model_type
2 active          c5ed16e8d0464810b84e679046358a78 NA      model_type
3 active          c5ed16e8d0464810b84e679046358a78 NA      model_type
4 active          c5ed16e8d0464810b84e679046358a78 NA      model_type
5 active          c5ed16e8d0464810b84e679046358a78 NA      model_type
  <NA>          tags                    
1 random_forest mlflow.user             
2 random_forest mlflow.source.name      
3 random_forest mlflow.runName          
4 random_forest mlflow.source.type      
5 random_forest mlflow.log-model.history
  <NA>                                                                                                                                                                                                                        
1 Admin                                                                                                                                                                                                                       
2 <console>                                                                                                                                                                                                                   
3 capricious-bug-290                                                                                                                                                                                                          
4 LOCAL                                                                                                                                                                                                                       
5 [{"run_id": "c5ed16e8d0464810b84e679046358a78", "artifact_path": "rf_custom_model", "utc_time_created": "2025-09-23 12:52:25.552819", "model_uuid": null, "flavors": {"crate": {"version": "0.1.0", "model": "crate.bin"}}}]

In [68]:
mlflow_end_run()

run_uuid                         experiment_id      run_name       user_id
1 26fdbc564fd34ffdb91de7eebf3a05c0 335984739251647557 bold-sheep-479 Admin  
2 26fdbc564fd34ffdb91de7eebf3a05c0 335984739251647557 bold-sheep-479 Admin  
3 26fdbc564fd34ffdb91de7eebf3a05c0 335984739251647557 bold-sheep-479 Admin  
4 26fdbc564fd34ffdb91de7eebf3a05c0 335984739251647557 bold-sheep-479 Admin  
  status   start_time          end_time           
1 FINISHED 2025-09-23 18:21:07 2025-09-23 18:21:49
2 FINISHED 2025-09-23 18:21:07 2025-09-23 18:21:49
3 FINISHED 2025-09-23 18:21:07 2025-09-23 18:21:49
4 FINISHED 2025-09-23 18:21:07 2025-09-23 18:21:49
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/26fdbc564fd34ffdb91de7eebf3a05c0/artifacts
2 mlflow-artifacts:/335984739251647557/26fdbc564fd34ffdb91de7eebf3a05c0/artifacts
3 mlflow-artifacts:/335984739251647557/26fdbc564fd34ffdb91de7eebf3a05c0/artifacts
4 mlflow-artifacts:/335984739251647557/26fdbc564fd34ffdb91de7eebf3a05c0/artifacts
  lifecycle_stage run_id                           metrics params    
1 active          26fdbc564fd34ffdb91de7eebf3a05c0 NA      model_type
2 active          26fdbc564fd34ffdb91de7eebf3a05c0 NA      model_type
3 active          26fdbc564fd34ffdb91de7eebf3a05c0 NA      model_type
4 active          26fdbc564fd34ffdb91de7eebf3a05c0 NA      model_type
  <NA>          tags               <NA>          
1 random_forest mlflow.user        Admin         
2 random_forest mlflow.source.name <console>     
3 random_forest mlflow.runName     bold-sheep-479
4 random_forest mlflow.source.type LOCAL

In [85]:
library(mlflow)
library(carrier)
library(randomForest)

# ---- Train model ----
set.seed(123)
model_rf <- randomForest(medv ~ ., data = train)

# ---- Evaluate ----
pred_rf <- predict(model_rf, newdata = test)
mse_rf <- mean((pred_rf - test$medv)^2)
r_squared_rf <- 1 - sum((pred_rf - test$medv)^2) / sum((test$medv - mean(test$medv))^2)

# ---- Start MLflow run ----
with(mlflow_start_run(), {

  mlflow_log_param("model_type", "random_forest")
  mlflow_log_metric("mse", mse_rf)
  mlflow_log_metric("r_squared", r_squared_rf)

  # Wrap model inside a crate (self-contained R object)
rf_crate <- crate(
  function(df, model_rf) {
    # Always call with namespace
    randomForest::predict(model_rf, newdata = df)
  },
  model_rf = model_rf
)

  # Save locally
  mlflow_save_model(rf_crate, path = "rf_model")

  # Log to MLflow tracking server
  mlflow_log_model(rf_crate, artifact_path = "model")
})


Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/25 16:09:43 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpMPGTdl/model to artifact_path=model


Root URI: mlflow-artifacts:/335984739251647557/ca99ed866cfc481689ed4c3c3f3aec08/artifacts



In [87]:
input <- data.frame(
  crim = 0.1,
  zn = 0,
  indus = 8.14,
  chas = 0,
  nox = 0.5,
  rm = 6.0,
  age = 65,
  dis = 4.0,
  rad = 4,
  tax = 307,
  ptratio = 18,
  black = 390,
  lstat = 12
)

predict(model_rf, input)

predict

1 
21.3535

function (object, ...) 
UseMethod("predict")
<bytecode: 0x000001eb63e3c1c8>
<environment: namespace:stats>

In [89]:
mlflow_list_registered_models(
)

ERROR: Error in mlflow_list_registered_models(): could not find function "mlflow_list_registered_models"


In [71]:
mlflow_end_run()

run_uuid                         experiment_id      run_name          
1 48d53e5d19ae415aa58f2373e1c8e96b 335984739251647557 marvelous-wasp-472
2 48d53e5d19ae415aa58f2373e1c8e96b 335984739251647557 marvelous-wasp-472
3 48d53e5d19ae415aa58f2373e1c8e96b 335984739251647557 marvelous-wasp-472
4 48d53e5d19ae415aa58f2373e1c8e96b 335984739251647557 marvelous-wasp-472
  user_id status   start_time          end_time           
1 Admin   FINISHED 2025-09-25 13:37:31 2025-09-25 13:41:59
2 Admin   FINISHED 2025-09-25 13:37:31 2025-09-25 13:41:59
3 Admin   FINISHED 2025-09-25 13:37:31 2025-09-25 13:41:59
4 Admin   FINISHED 2025-09-25 13:37:31 2025-09-25 13:41:59
  artifact_uri                                                                   
1 mlflow-artifacts:/335984739251647557/48d53e5d19ae415aa58f2373e1c8e96b/artifacts
2 mlflow-artifacts:/335984739251647557/48d53e5d19ae415aa58f2373e1c8e96b/artifacts
3 mlflow-artifacts:/335984739251647557/48d53e5d19ae415aa58f2373e1c8e96b/artifacts
4 mlflow-artifacts:/335984739251647557/48d53e5d19ae415aa58f2373e1c8e96b/artifacts
  lifecycle_stage run_id                           metrics   <NA>      
1 active          48d53e5d19ae415aa58f2373e1c8e96b mse       11.8035513
2 active          48d53e5d19ae415aa58f2373e1c8e96b r_squared  0.8601192
3 active          48d53e5d19ae415aa58f2373e1c8e96b mse       11.8035513
4 active          48d53e5d19ae415aa58f2373e1c8e96b r_squared  0.8601192
  <NA>                <NA> <NA>                             params    
1 2025-09-25 13:37:32 0    48d53e5d19ae415aa58f2373e1c8e96b model_type
2 2025-09-25 13:37:32 0    48d53e5d19ae415aa58f2373e1c8e96b model_type
3 2025-09-25 13:37:32 0    48d53e5d19ae415aa58f2373e1c8e96b model_type
4 2025-09-25 13:37:32 0    48d53e5d19ae415aa58f2373e1c8e96b model_type
  <NA>          tags               <NA>              
1 random_forest mlflow.user        Admin             
2 random_forest mlflow.source.name <console>         
3 random_forest mlflow.runName     marvelous-wasp-472
4 random_forest mlflow.source.type LOCAL